<a href="https://colab.research.google.com/github/Gabriellemga/Python_Cursos_Alura/blob/main/Python_analise_dados_com_sql/Projeto_com_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01: selecionando colunas, agrupando e contando

Digamos que os DataFrames tenham os mesmos nomes de cada uma das tabelas, vamos então ver como seria o código em Pandas que corresponderia ao resultado obtido pelas consultas que fizemos em aula no SQL.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
url_itens_pedidos = 'https://github.com/alura-cursos/SQL-python-integracao/raw/main/TABELAS/itens_pedidos.csv'
url_pedidos = 'https://github.com/alura-cursos/SQL-python-integracao/raw/main/TABELAS/pedidos.csv'
url_produto = 'https://github.com/alura-cursos/SQL-python-integracao/raw/main/TABELAS/produtos.csv'
url_vendedores = 'https://github.com/alura-cursos/SQL-python-integracao/raw/main/TABELAS/vendedores.csv'

In [4]:
itens_pedidos = pd.read_csv(url_itens_pedidos)
pedidos = pd.read_csv(url_pedidos)
produtos = pd.read_csv(url_produto)
vendedores = pd.read_csv(url_vendedores)

## Parte 1: selecionar coluna Condicao

Podemos selecionar a coluna Condicao e executar a transformação da Series para DataFrame com pd.DataFrame

In [5]:
pd.DataFrame(produtos['Condicao'])

,Condicao
0,Usado
1,Usado
2,Usado
3,Usado
4,Usado
...,...
200,Usado
201,Usado
202,Usado
203,Usado


## Parte 2: contar as frequências

Para aplicar uma função de agregação por contagem de frequências, podemos aplicar groupby e a agregação de contagem com size. O nome da coluna de 'Quantidade' pode ser definido com reset_index.

In [6]:
produtos.groupby('Condicao').size().reset_index(name='Quantidade')

,Condicao,Quantidade
0,Novo com etiqueta,22
1,Novo sem etiqueta,7
2,Usado,176


# 02: selecionando e relacionando dados

## Parte 1: selecionar os dados

Na consulta, lemos todos os dados das tabelas produtos e itens_pedidos. Para fazer isso em Python com Pandas, basta usar o objeto específico do DataFrame.

In [7]:
produtos.head(3)

,produto_id,produto,preco,marca,sku,Condicao
0,21244,Oculos Lente Azulada,1120,D&g Dolce & Gabbana,209297,Usado
1,9981,Bolsa Coral Saco,4000,Givenchy,278612,Usado
2,84176,Camisa Xadrez Verde,310,Joe Fresh,322482,Usado


In [8]:
itens_pedidos.head(3)

,id_nf,produto_id,pedido_id,quantidade,valor_unitario,valor_total,Estado,frete
0,1,41518,341,3,260,780,BR-BA,156.0
1,2,4307,1174,5,6175,30875,BR-RJ,6175.0
2,3,22407,1399,3,200,600,BR-PB,120.0


## Parte 2: relacionar os produtos por quantidade

Para um código equivalente em Python com Pandas, começamos mesclando com merge os DataFrames itens_pedidos e produtos, com base na coluna produto_id. Em seguida, aplicamos a função groupby() para agrupar os dados pelo nome do produto e depois, usamos a função sum() para obter a soma da coluna quantidade para cada grupo. Por fim, para ordenar o resultado em ordem ascendente de acordo com a quantidade, usamos a função sort_values():

In [9]:
df_prod_quant = itens_pedidos.merge(produtos, on='produto_id')
df_prod_quant = df_prod_quant.groupby('produto')['quantidade'].sum().reset_index()
df_prod_quant = df_prod_quant.sort_values('quantidade', ascending=True).reset_index(drop=True)
df_prod_quant

,produto,quantidade
0,Vestido Malha Estampa,318
1,Blusa Listrada Malha,322
2,Blusa Malha Azul,325
3,Blusa Babados Off-white,337
4,Blusa Cinza,341
...,...,...
196,Saia Midi Cinto,549
197,Calca Alfaiataria Preta,750
198,Saia Pregas Preta,751
199,Calca Jeans Costuras,755


# 03: filtro em dado textual

Continuamos considerando que temos DataFrames com nomes correspondentes às tabelas do nosso conjunto de dados.

Para essa consulta, iniciamos filtrando os pedidos que ocorreram em 2020 com a função str.startswith('2020'), que filtra os pedidos que possuem a data de compra iniciando com o valor '2020'. Depois, mesclamos o DataFrame filtrado (df_filtered) com o DataFrame de vendedores com base na coluna 'vendedor_id' através da função merge.

Em seguida, usamos a função groupby, para agrupar os dados pelo nome do vendedor ('nome_vendedor') e, em seguida, calculamos a média da coluna 'total' para cada grupo. Antes de concluir, ordenamos os resultados em ordem descrescente com base na coluna 'total' usando a função sort_values(), e também resetamos os index com reset_index. Por fim, alteramos o nome da coluna 'total' para 'Valor médio por vendas', correspondendo ao resultado no SQL:

In [10]:
df_filtrado = pedidos[pedidos['data_compra'].str.startswith('2020')]
df_mesclado = df_filtrado.merge(vendedores, on='vendedor_id')
df_agrupado = df_mesclado.groupby('nome_vendedor')['total'].mean().reset_index()
df_filtro_data = df_agrupado.sort_values('total', ascending=False).reset_index(drop=True)
df_filtro_data.columns = ['nome_vendedor', 'Valor médio por vendas']
df_filtro_data

,nome_vendedor,Valor médio por vendas
0,Daniel Siqueira,1972.338526
1,Millena Pereira,1894.614343
2,Ana Duarte,1843.452582
3,Nadia Oliveira,1818.342200
4,Paulo Calanca,1813.469855


# 04: contagem por estados

Para essa consulta, começamos com a função groupby() para agrupar os dados pelo estado (coluna 'Estado') e, em seguida, aplicamos a função size() para obter a contagem de pedidos para cada estado. A função reset_index(name='Pedidos') redefine o índice do resultado agrupado e renomeia a coluna de contagem para 'Pedidos'. O resultado é um DataFrame com duas colunas: 'Estado' e 'Pedidos', onde 'Pedidos' representa a contagem de pedidos para cada estado.

Depois, usamos a função sort_values() para ordenar o DataFrame resultado em ordem decrescente com base na coluna 'Pedidos'. Ao definir ascending=False, garantimos que a ordenação seja feita em ordem decrescente. Isso nos dá o DataFrame final df_estados, correspondente ao resultado obtido na consulta:

In [11]:
df_agrupados = itens_pedidos.groupby('Estado').size().reset_index(name='Pedidos')
df_estados = df_agrupados.sort_values('Pedidos', ascending=False).reset_index(drop=True)
df_estados

,Estado,Pedidos
0,BR-MA,974
1,BR-DF,953
2,BR-MT,937
3,BR-GO,933
4,BR-PA,932
5,BR-AL,928
6,BR-PE,927
7,BR-RO,925
8,BR-RR,925
9,BR-RN,921


# 05: agrupando dados

Iniciamos com a função merge() para combinar os DataFrames pedidos, vendedores e itens_pedidos. Depois, aplicamos a função query() para selecionar apenas os itens de pedidos com estado de São Paulo ('BR-SP'). Em seguida, utilizamos groupby() para agrupar por nome da pessoa vendedora e aplicamos size() para obter a contagem de vendas para cada pessoa vendedora. Por fim, aplicamos sort_values() para ordenar o DataFrame resultante em ordem decrescente com base na quantidade de vendas:

In [12]:
df_unido = pedidos.merge(vendedores, on='vendedor_id').merge(itens_pedidos, on='pedido_id')
df_filtrado = df_unido.query("Estado == 'BR-SP'")
df_vendedores_sp = df_filtrado.groupby('nome_vendedor').size().reset_index(name='quantidade_vendas')
df_vendedores_sp = df_vendedores_sp.sort_values('quantidade_vendas', ascending=False)
df_vendedores_sp

,nome_vendedor,quantidade_vendas
1,Daniel Siqueira,190
4,Paulo Calanca,182
3,Nadia Oliveira,181
0,Ana Duarte,180
2,Millena Pereira,157
